- try fold 7

In [1]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd 
from sklearn import preprocessing
from sklearn.metrics import log_loss, mean_squared_error
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_selection import VarianceThreshold

sys.path.append('../input/multilabelstraifier/')
sys.path.append('../input/lookahead/')
from ml_stratifiers import MultilabelStratifiedKFold
from lookahead import Lookahead
warnings.filterwarnings('ignore')

import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf

In [2]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
non_targets = pd.read_csv(DATA_DIR + 'train_targets_nonscored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [3]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

In [4]:
# to remove outlier paienr
# original_remove_index = [16674] # remove_index = [15361]

In [5]:
noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
cons_train_index = train[train.cp_type!="ctl_vehicle"].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

# preprocess

In [6]:
train = train[train.index.isin(cons_train_index)].copy().reset_index(drop=True)
targets = targets[targets.index.isin(cons_train_index)].copy().reset_index(drop=True)
non_targets = non_targets[non_targets.index.isin(cons_train_index)].copy().reset_index(drop=True)

In [7]:
#tmp = train.iloc[15361, :].copy().drop(["sig_id", "cp_type", "cp_time", "cp_dose"])
#tmp1 = np.array(tmp)
#np.sum(tmp1 == 10), np.sum(tmp1==-10)

In [8]:
non_target_feats = [i for i in non_targets.columns if i != "sig_id"]
nontarget_dists = pd.DataFrame(np.sum(non_targets[non_target_feats])).reset_index(drop=False)
nontarget_dists.columns = ["target", "number"]
nontarget_dists = nontarget_dists.sort_values("number", ascending=False).reset_index(drop=True)
drop_list1 = list(nontarget_dists[nontarget_dists.number==0]["target"].values)
print("first drop", len(drop_list1))
non_targets.drop(drop_list1, axis=1, inplace=True)
print("shape after 1st drop:", non_targets.shape)
non_target_feats = [i for i in non_targets.columns if i != "sig_id"]
print(len(non_target_feats))

first drop 71
shape after 1st drop: (21948, 332)
331


# feature engineering

In [9]:
def fe(df):
    tmp = df.copy()
    #tmp['g_sum'] = tmp[g_feats].sum(axis = 1)
    #tmp['g_mean'] = tmp[g_feats].mean(axis = 1)
    #tmp['g_std'] = tmp[g_feats].std(axis = 1)
    #tmp['g_kurt'] = tmp[g_feats].kurtosis(axis = 1)
    #tmp['g_skew'] = tmp[g_feats].skew(axis = 1)
    #tmp['c_sum'] = tmp[c_feats].sum(axis = 1)
    #tmp['c_mean'] = tmp[c_feats].mean(axis = 1)
    #tmp['c_std'] = tmp[c_feats].std(axis = 1)
    #tmp['c_kurt'] = tmp[c_feats].kurtosis(axis = 1)
    #tmp['c_skew'] = tmp[c_feats].skew(axis = 1)
    #tmp['gc_sum'] = tmp[c_feats + g_feats].sum(axis = 1)
    #tmp['gc_mean'] = tmp[c_feats + g_feats].mean(axis = 1)
    #tmp['gc_std'] = tmp[c_feats + g_feats].std(axis = 1)
    #tmp['gc_kurt'] = tmp[c_feats + g_feats].kurtosis(axis = 1)
    #tmp['gc_skew'] = tmp[c_feats + g_feats].skew(axis = 1)
    #tmp.loc[:, 'cp_dose'] = tmp.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})        
    tmp.drop(["cp_type", "sig_id", "cp_dose", "cp_time"], axis=1, inplace=True)
    #col = list(tmp.columns)
    #col = col[1:] + col[:1]
    #tmp = tmp[col]
    return tmp

f_train = fe(train)
f_test = fe(test)

print(f_train.shape, f_test.shape)

(21948, 872) (3982, 872)


In [10]:
fn_train = f_train.copy().to_numpy()
fn_test = f_test.copy().to_numpy()

ss = preprocessing.RobustScaler()
fn_train= ss.fit_transform(fn_train)
fn_test = ss.transform(fn_test)

#fn_nontargets = non_targets.drop("sig_id", axis=1).copy().to_numpy()
fn_targets = targets.drop("sig_id", axis=1).copy().to_numpy()

In [11]:
class myLSTM(nn.Module):
    def __init__(self, last_num):
        super().__init__()

        self.lstm_hidden_size = 772
        self.c_lstm_hidden_size = 100
        self.g_layer_num = 1
        self.c_layer_num = 1
        
        self.hidden_dim = 512
        self.hidden_dim_c = 10
        
        self.lstm = nn.LSTM(self.lstm_hidden_size, self.hidden_dim, batch_first=True, bidirectional=True, num_layers=self.g_layer_num)
        self.c_lstm = nn.LSTM(self.c_lstm_hidden_size, self.hidden_dim_c, batch_first=True, bidirectional=True, num_layers=self.c_layer_num)

        self.dropout = nn.Dropout(0.1)
        self.out = nn.utils.weight_norm(nn.Linear((self.hidden_dim+self.hidden_dim_c) * 2 , last_num))
        self.batch_norm = nn.BatchNorm1d((self.hidden_dim+self.hidden_dim_c) * 2)

    def forward(self, cont_g, cont_c): 
        cont_g = torch.unsqueeze(cont_g, 1)
        h_lstm, lstm_out = self.lstm(cont_g) # h_lstm: 256 * 1 * (2 * 512)
        conc_g = h_lstm.view(-1, self.hidden_dim * 2)
        
        cont_c = torch.unsqueeze(cont_c, 1)
        h_lstm_c, lstm_out_c = self.c_lstm(cont_c) # h_lstm: 256 * 1 * (2 * 5)
        conc_c = h_lstm_c.view(-1, self.hidden_dim_c * 2)
        
        conc = torch.cat((conc_g, conc_c),1)
        conc = self.batch_norm(conc)
        
        dropped = self.dropout(conc)
        out = self.out(dropped)
        return out

In [12]:
batch_size = 256
train_epochs = 30
n_folds=7
EARLY_STOPPING_STEPS = 10

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
def mean_log_loss(y_true, y_pred):
    metrics = []
    for i, target in enumerate(target_feats):
        metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float), labels=[0,1]))
    return np.mean(metrics)

def seed_everything(seed=1234): 
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

def modelling_lstm(tr, target, te, sample_seed, last_num):
    
    mskf=MultilabelStratifiedKFold(n_splits = n_folds, shuffle=True, random_state=2)
    
    seed_everything(seed=sample_seed) 
    X_train = tr.copy()
    y_train = target.copy()
    X_test = te.copy()
    test_len = X_test.shape[0]
    
    models = []
    
    X_test_g = torch.tensor(X_test[:,:772], dtype=torch.float32)
    X_test_c = torch.tensor(X_test[:,772:872], dtype=torch.float32)

    X_test = torch.utils.data.TensorDataset(X_test_g, X_test_c) 
    test_loader = torch.utils.data.DataLoader(X_test, batch_size=batch_size, shuffle=False)
    
    oof = np.zeros([len(X_train),y_train.shape[1]])
    oof_targets = np.zeros([len(X_train),y_train.shape[1]])
    pred_value = np.zeros([test_len, y_train.shape[1]])
    scores = []
    
    for fold, (train_index, valid_index) in enumerate(mskf.split(X_train, y_train)):
        print("Fold "+str(fold+1))
        X_train2_g = torch.tensor(X_train[train_index,:772], dtype=torch.float32)
        X_valid2_g = torch.tensor(X_train[valid_index,:772], dtype=torch.float32)
        X_train2_c = torch.tensor(X_train[train_index,772:872], dtype=torch.float32)
        X_valid2_c = torch.tensor(X_train[valid_index,772:872], dtype=torch.float32)
        
        y_train2 = torch.tensor(y_train[train_index], dtype=torch.float32)
        y_valid2 = torch.tensor(y_train[valid_index], dtype=torch.float32)
            
        clf = myLSTM(last_num)
        loss_fn = torch.nn.BCEWithLogitsLoss() 

        optimizer = optim.Adam(clf.parameters(), lr = 0.01, weight_decay=1e-5) 
        #lookahead = Lookahead(optimizer, k=10, alpha=0.6) #lookahead
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, eps=1e-4, verbose=True)
        
        train = torch.utils.data.TensorDataset(X_train2_g, X_train2_c, y_train2)
        valid = torch.utils.data.TensorDataset(X_valid2_g, X_valid2_c, y_valid2)
        
        clf.to(device)
        
        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True) 
        valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
        
        best_val_loss = np.inf
        stop_counts = 0
        for epoch in range(train_epochs):
            start_time = time.time()
            clf.train()
            avg_loss = 0.
            for x_batch_g, x_batch_c, y_batch in tqdm(train_loader, disable=True):
                x_batch_g = x_batch_g.to(device)
                x_batch_c = x_batch_c.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch_g, x_batch_c) 
                loss = loss_fn(y_pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                avg_loss += loss.item() / len(train_loader)        
            
            clf.eval()
            avg_val_loss = 0.
            for i, (x_batch_g, x_batch_c, y_batch) in enumerate(valid_loader): 
                x_batch_g = x_batch_g.to(device)
                x_batch_c = x_batch_c.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch_g, x_batch_c).detach()
                avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
                
            elapsed_time = time.time() - start_time 
            scheduler.step(avg_val_loss)
                    
            if avg_val_loss < best_val_loss:
                stop_counts = 0
                best_val_loss = avg_val_loss
                print('Best model: Epoch {} \t loss={:.6f} \t val_loss={:.6f} \t time={:.2f}s'.format(
                    epoch + 1, avg_loss, avg_val_loss, elapsed_time))
                torch.save(clf.state_dict(), 'best-model-parameters.pt')

            else:
                stop_counts += 1
        
            #if stop_counts >= EARLY_STOPPING_STEPS: 
            #    break
         
        pred_model = myLSTM(last_num)
        pred_model.load_state_dict(torch.load('best-model-parameters.pt'))
        pred_model.eval()
        
        # validation check ----------------
        oof_epoch = np.zeros([X_valid2_g.size(0), y_train.shape[1]])
        target_epoch = np.zeros([X_valid2_g.size(0), y_train.shape[1]])
        for i, (x_batch_g, x_batch_c, y_batch) in enumerate(valid_loader): 
                y_pred = pred_model(x_batch_g, x_batch_c).sigmoid().detach()
                oof_epoch[i * batch_size:(i+1) * batch_size,:] = y_pred.cpu().numpy()
                target_epoch[i * batch_size:(i+1) * batch_size,:] = y_batch.cpu().numpy()
        print("Fold {} log loss: {}".format(fold+1, mean_log_loss(target_epoch, oof_epoch)))
        scores.append(mean_log_loss(target_epoch, oof_epoch))
        oof[valid_index,:] = oof_epoch
        oof_targets[valid_index,:] = target_epoch
        #-----------------------------------
        
        # test predcition --------------
        test_preds = np.zeros([test_len, y_train.shape[1]])
        for i, (x_batch_g, x_batch_c, ) in enumerate(test_loader): 
            y_pred = pred_model(x_batch_g, x_batch_c).sigmoid().detach()
            test_preds[i * batch_size:(i+1) * batch_size, :] = y_pred.cpu().numpy()
        pred_value += test_preds / n_folds
        # ------------------------------
        
    print("Seed {}".format(seed_))
    for i, ele in enumerate(scores):
        print("Fold {} log loss: {}".format(i+1, scores[i]))
    print("Std of log loss: {}".format(np.std(scores)))
    print("Total log loss: {}".format(mean_log_loss(oof_targets, oof)))

    return oof, oof_targets, pred_value

cuda


In [13]:
#print("Transfer learning")
#for seed_ in [0]:
#    files = modelling_lstm(fn_train, fn_all_targets, fn_test, seed_,  fn_non_targets.shape[1])

target_oof = np.zeros([len(fn_train),fn_targets.shape[1]])
target_pred = np.zeros([len(fn_test),fn_targets.shape[1]])

seeds = [0, 10, 40]

print("Final prediction")
for seed_ in seeds:
    oof, oof_targets, pytorch_pred = modelling_lstm(fn_train, fn_targets, fn_test, seed_, fn_targets.shape[1])
    target_oof += oof / len(seeds)
    target_pred += pytorch_pred / len(seeds)
print("Total log loss in targets: {}".format(mean_log_loss(oof_targets, target_oof)))

Final prediction
Fold 1
Best model: Epoch 1 	 loss=0.135719 	 val_loss=0.019513 	 time=0.88s
Best model: Epoch 2 	 loss=0.018417 	 val_loss=0.018191 	 time=0.72s
Best model: Epoch 3 	 loss=0.017330 	 val_loss=0.017870 	 time=0.74s
Best model: Epoch 4 	 loss=0.016946 	 val_loss=0.017520 	 time=0.72s
Best model: Epoch 5 	 loss=0.016662 	 val_loss=0.017308 	 time=0.72s
Best model: Epoch 6 	 loss=0.016509 	 val_loss=0.017236 	 time=0.74s
Best model: Epoch 7 	 loss=0.016423 	 val_loss=0.017228 	 time=0.73s
Epoch    11: reducing learning rate of group 0 to 1.0000e-03.
Best model: Epoch 12 	 loss=0.014612 	 val_loss=0.016319 	 time=0.75s
Best model: Epoch 13 	 loss=0.013275 	 val_loss=0.016228 	 time=0.95s
Epoch    17: reducing learning rate of group 0 to 1.0000e-04.
Fold 1 log loss: 0.01641108022170241
Fold 2
Best model: Epoch 1 	 loss=0.139104 	 val_loss=0.019464 	 time=0.71s
Best model: Epoch 2 	 loss=0.018421 	 val_loss=0.017866 	 time=1.07s
Best model: Epoch 3 	 loss=0.017428 	 val_loss=

In [14]:
t = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
train_checkscore = t.copy()
train_checkscore.loc[train_checkscore.index.isin(cons_train_index),target_feats] = target_oof
train_checkscore.loc[train_checkscore.index.isin(noncons_train_index),target_feats] = 0

t.drop("sig_id", axis=1, inplace=True)

print('OOF log loss: ', log_loss(np.ravel(t), np.ravel(np.array(train_checkscore.iloc[:,1:]))))

OOF log loss:  0.014890611617043806


In [15]:
sub[target_feats] = target_pred
sub.loc[noncons_test_index,target_feats] = 0
sub.to_csv('submission.csv', index=False)